In [ ]:
import orjson, os
with open("local.settings.json") as f:
    os.environ.update(orjson.loads(f.read())["Values"])

# Database

In [ ]:
from sqlalchemy import create_engine
engine = create_engine(os.environ['DATABIND_SQL_KEYSTONE'])
schema = 'keystone'
table_name = 'Audience'
columns = []

In [ ]:
from sqlalchemy import MetaData, Table, select
import pandas as pd

# Create a MetaData object
metadata = MetaData()

# Reflect the table
table = Table(table_name, metadata, schema=schema, autoload_with=engine)

# select
columns = ["id", "status"]
selected_columns = [table.c[col] for col in columns]
stmt = select(*selected_columns)

# Execute the query
with engine.connect() as conn:
    result = conn.execute(stmt)
    # Use mappings() to fetch rows as dictionaries
    rows = [row for row in result.mappings()]

audiences = pd.DataFrame(rows)

# Table

In [ ]:
import os
import json
from azure.data.tables import TableClient
import pandas as pd

def get_partition_keys_from_connection_string(
    conn_str: str,
    name_filter: str = "orchestrator_esquire_audience"
) -> pd.DataFrame:
    """
    Returns a DataFrame of entities with PartitionKey and Name
    where Name == name_filter. Filtering is applied server-side,
    and results are ordered by CreatedTime ascending (oldest first).

    :param conn_str: Azure Storage connection string
    :param name_filter: The value to match in the 'Name' field
    :return: pandas DataFrame with PartitionKey, Name, RuntimeStatus, CustomStatus, CreatedTime
    """
    # Initialize client
    table_client = TableClient.from_connection_string(conn_str, table_name="productionInstances")

    # Select minimal columns to reduce payload
    select_fields = ["PartitionKey", "RuntimeStatus", "CustomStatus", "CreatedTime"]

    # OData filter: property names are case-sensitive to your schema
    odata_filter = f"Name eq '{name_filter}'"

    # Query entities (pagination handled internally)
    entities = list(table_client.query_entities(select=select_fields, query_filter=odata_filter))

    # Convert to DataFrame
    df = pd.DataFrame(entities)

    # Parse JSON in CustomStatus safely
    if "CustomStatus" in df.columns:
        df["CustomStatus"] = df["CustomStatus"].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

    # Sort by CreatedTime (ascending, oldest first)
    if "CreatedTime" in df.columns:
        df = df.sort_values(by="CreatedTime", ascending=True).reset_index(drop=True)

    return df


# Compare

In [ ]:
instances = get_partition_keys_from_connection_string(os.getenv("AzureWebJobsStorage", ""))
instance_ids = set(instances["PartitionKey"].to_list())
audience_ids = set(audiences[audiences["status"]]["id"].to_list())
absent_audiences = instance_ids - audience_ids
display(len(instance_ids & audience_ids))
display(len(absent_audiences))
display(f"{len(absent_audiences) / len(instance_ids):.2%} set to be removed")

# Delete

In [ ]:
import requests

for id in absent_audiences:
    print(id)
    resp = requests.delete(
        url="https://esquire-auto-audience.azurewebsites.net/runtime/webhooks/durabletask/instances/{id}".format(
            id=id
        ),
        params={
            "taskHub": "production",
            "connection": "Storage",
            "code": os.environ['AZFUNC_MASTER_CODE'],
        },
    )


In [ ]:
def has_error_state(v) -> bool:
    # v can be a dict, JSON string, or NaN
    if isinstance(v, dict):
        return v.get("state") == "Error"
    if pd.isna(v):
        return False
    try:
        obj = json.loads(v)
        return isinstance(obj, dict) and obj.get("state") == "Error"
    except Exception:
        return False

mask_error = instances["CustomStatus"].apply(has_error_state)

instances = get_partition_keys_from_connection_string(
    os.getenv("AzureWebJobsStorage", "")
)
failed_ids = instances[
    (instances["RuntimeStatus"] == "Failed")
    # & (
    #     ~instances["PartitionKey"].isin(
    #         [
    #             "cmea1702f0010twbajoqog052",
    #             "cmea0y34i000wtwba46i6zsgu",
    #             "cmg9hmvpx000shfz8s14f2rha",
    #             "cmggn76i4000avub2r1hl5c0r",
    #             "cmg6voslk001rhia2p20bjswr",
    #             "cly1z4etl00232144s3gwe6wm",
    #             "clz1iu5i2000011il1gcvofz6",
    #             "clzbmml1g002xern43pnf0b9k",
    #             "clzojel6m001pzgkp1hcrbn5p",
    #             "cm3ouvcd0001aze18wmapnmx8",
    #             "cm4vtqklg00581103ugyr6v7z",
    #             "cm5mwchc0001kbb7961qxkvyo",
    #             "cm4vo1apu004n1103v4j4bhmp",
    #             "cm3j6t1xo000jx2ycxdqgtmwn",
    #             "cm3sxuzxl000zovfjv79satyl",
    #             "cm7utgax2003ht62990n4cgkj",
    #             "cmfzi7usj001e7rt0mnebygfb",
    #             "cmfzhq2n3001t13vjw235hxql",
    #             "cmg5igyfy00002ihbwtgf3q5t",
    #             "cmg5eapst001o146yas6l9ytp",
    #             "cmfwylljd001213vjqaoyhlq6",
    #             "cmfvjzn4s000j13vjq9vjegmh",
    #             "cmfebpmkq0000rmszmzely63n",
    #             "cmfwyut25000u7rt00g22amjj",
    #             "cmfzijir4001i7rt0g5duzv0t",
    #             "cmg5id4le0000vv796gmd1suf",
    #             "cmfo9uykn000axprcmk6e52e3",
    #             "cmfv6l5ge00087rt0babmbxji",
    #         ]
    #     )
    # )
    # & ~mask_error
]["PartitionKey"].to_list()[0:25]

for id in failed_ids:
    print(id)
    requests.delete(
        url="https://esquire-auto-audience.azurewebsites.net/runtime/webhooks/durabletask/instances/{id}".format(
            id=id
        ),
        params={
            "taskHub": "production",
            "connection": "Storage",
            "code": os.environ["AZFUNC_MASTER_CODE"],
        },
    )
    requests.post(
        url="https://esquire-auto-audience.azurewebsites.net/api/audiences/{id}".format(
            id=id
        ),
        params={
            "force": 1,
            "code": os.environ["AZFUNC_MASTER_CODE"],
        },
    )